## 🎯 Practice Exercises
## Exercise 1: Build Your Own Agentic RAG System

### Task
Build an agentic RAG system on a topic of your choice (NOT biochemistry - use your own domain).

### Domain Suggestions
- Technology tutorials (Python, JavaScript, etc.)
- Agriculture
- Finance
- Historical documents
- Study notes from a course
- Recipe collection
- Legal documents (simplified)

### Requirements

**1. Document Collection**
- Gather 5-10 documents (PDF or TXT) in your chosen domain
- Documents should be substantial (5000+ words each)
- Topics should be related but distinct

**2. Vector Store Setup**
- Load documents using appropriate loader
- Split into chunks (experiment with chunk size)
- Create Chroma vector store with embeddings
- Test retrieval with sample queries

**3. Retrieval Tool**
- Create `@tool` decorated function for retrieval
- Use MMR or similarity search
- Return formatted context
- Include metadata in responses

**4. Agentic RAG System**
- Build LangGraph with agent and tool nodes
- Implement conditional edges (agent decides when to retrieve)
- Add conversation memory
- Create helpful system prompt

**5. Testing & Evaluation**
- Test with 10 diverse queries:
  - 5 that require retrieval
  - 5 that don't require retrieval
- Document which queries trigger retrieval
- Evaluate answer quality

### Deliverables
1. Jupyter notebook with complete implementation
2. Test results showing agent decisions
3. Brief report (300-500 words):
   - What domain did you choose and why?
   - How did you tune chunk size?
   - Did the agent make good retrieval decisions?
   - What worked well? What needs improvement?

### Example System Behavior
```
Query: "What is Python?"
Agent Decision: NO retrieval needed (general knowledge)
Response: "Python is a high-level programming language..."

Query: "How do I use the new API endpoint for user authentication?"
Agent Decision: YES, retrieve from documentation
Response: [Retrieved docs] "Based on the documentation, the new authentication endpoint..."
```


In [3]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import glob
import warnings
warnings.filterwarnings("ignore")
import os

print("✅ All imports successful")

✅ All imports successful


In [4]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [5]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [6]:
documents = []

for pdf_path in glob.glob("documents/*.pdf"):  #
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    documents.extend(docs)

print(f"Loaded {len(documents)} PDF Documents.")

Loaded 1092 PDF Documents.


In [7]:
# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Characters per chunk
    chunk_overlap=100,     # Overlap to preserve context
    separators=["\n\n", "\n", ".", " ", ""]
)

# Split documents
doc_splits = text_splitter.split_documents(documents)

print(f"✅ Created {len(doc_splits)} chunks")
print(f"\nSample chunk:")
print(f"{doc_splits[0].page_content[:200]}...")

✅ Created 255 chunks

Sample chunk:
LECTURE NOTES ON  
ELECTRICAL 
MACHINE-II 
Subject Code - BEE 1401 
For B-Tech 4th SEM EE & EEE 
[Part-II] 
[Module-III & IV] 
  
 
 
 
 
 
 
 
 
 
 
 
VEER SURENDRA SAI UNIVERSITY OF TECHNOLOGY 
Depa...


In [8]:
# Initialize embeddings (using OpenAI)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=openai_api_key
)

print("✅ Embeddings model initialized")

✅ Embeddings model initialized


In [9]:
# Create Chroma vector store
chroma_path = "./chroma_db_elec_agentic_rag"

# Create vector store from documents
vectorstore = Chroma(
    collection_name="elec_agentic_rag_docs",
    persist_directory=chroma_path,
    embedding_function=embeddings
)

# Add documents
vectorstore.add_documents(documents=doc_splits)

print(f"✅ Vector store created with {len(doc_splits)} chunks")
print(f"   Persisted to: {chroma_path}")

✅ Vector store created with 255 chunks
   Persisted to: ./chroma_db_elec_agentic_rag


In [10]:
# Test the vector store
test_query = "What is nulling procedure for output voltage?"
test_results = vectorstore.similarity_search(test_query, k=2)

print(f"Query: {test_query}")
print(f"\nTop result:")
print(f"{test_results[0].page_content[:200]}...")
print(f"\n✅ Retrieval working!")

Query: What is nulling procedure for output voltage?

Top result:
11
Fig.3.6:TypicalCircuitstominimizeerrorinoutputvoltageduetoinputoffsetvoltage&offset
current.
2.3.1.4 Nulling Procedure For Output Voltage
1. Buildthecircuit&include(a)theCCR
(b)Voltageoffsetnullcir...

✅ Retrieval working!


In [11]:
@tool
def retrieve_documents(query: str) -> str:
    """
    Search for relevant documents in the knowledge base.
    
    Use this tool when you need information from the document collection
    to answer the user's question. Do NOT use this for:
    - General knowledge questions
    - Greetings or small talk
    - Simple calculations
    
    Args:
        query: The search query describing what information is needed
        
    Returns:
        Relevant document excerpts that can help answer the question
    """
    # Use MMR (Maximum Marginal Relevance) for diverse results
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 10}
    )
    
    # Retrieve documents
    results = retriever.invoke(query)
    
    if not results:
        return "No relevant documents found."
    
    # Format results
    formatted = "\n\n---\n\n".join(
        f"Document {i+1}:\n{doc.page_content}"
        for i, doc in enumerate(results)
    )
    
    return formatted

print("✅ Retrieval tool created")

✅ Retrieval tool created


In [12]:
# Test tool directly
test_result = retrieve_documents.invoke({"query": "What is synchronous speed?"})
print(f"Tool result (first 300 chars):\n{test_result[:300]}...")

Tool result (first 300 chars):
Document 1:
Lecture Notes –  Electrical Machine - II [BEE 1401]           Page  | 93 
 
4.5.2 Speed Control 
Speed of Schrage Motor can be obtained above and below Synchronous speed by changing the 
Brush position i.e. changing “θ”  (‘θ’ – Brush separation angle). 
In Fig: 4.22 (a) Brush pair on the...


In [13]:
system_prompt = SystemMessage(content="""You are a helpful assistant with access to a document retrieval tool.

RETRIEVAL DECISION RULES:

DO NOT retrieve for:
- Greetings: "Hello", "Hi", "How are you"
- Questions about your capabilities: "What can you help with?", "What do you do?"
- Simple math or general knowledge: "What is 2+2?"
- Casual conversation: "Thank you", "Goodbye"

DO retrieve for:
- Questions asking for specific information that would be in documents
- Requests for facts, definitions, or explanations about specialized topics in Electrical machine, Electronics.
- Any question where citing sources would improve the answer

Rule of thumb: If the user is asking for information (not just chatting), retrieve first.

When you retrieve documents, cite them in your answer. If documents don't contain the answer, say so.
""")

print("✅ System prompt configured")

✅ System prompt configured


In [14]:
# Bind tool to LLM
tools = [retrieve_documents]
llm_with_tools = llm.bind_tools(tools)

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to retrieve or answer directly.
    """
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide whether to call tools or finish.
    """
    last_message = state["messages"][-1]
    
    if last_message.tool_calls:
        return "tools"
    return "__end__"

print("✅ Agent nodes defined")

✅ Agent nodes defined


In [15]:
# Build graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

# Add memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agentic RAG system compiled")

✅ Agentic RAG system compiled


In [17]:
# Visualize the agentic RAG graph
try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Graph: START → assistant → [if tool_call] → tools → assistant → END")

Could not display graph: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`
Graph: START → assistant → [if tool_call] → tools → assistant → END


In [18]:
def query_agent(user_input: str, thread_id: str = "default_session"):
    """
    Improved query function with clearer output.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )

    # Check what happened
    used_retrieval = False
    final_answer = None

    for message in result["messages"]:
        if isinstance(message, AIMessage):
            if message.tool_calls:
                used_retrieval = True
                print(f"🔍 Agent: [Calling retrieval tool...]")
            if message.content and not message.tool_calls:
                final_answer = message.content

    # Always print final answer
    if final_answer:
        print(f"🤖 Agent: {final_answer}")
    else:
        print(f"⚠️ No response generated after retrieval!")

    # Summary
    print(f"\n📊 Decision: {'USED RETRIEVAL' if used_retrieval else 'ANSWERED DIRECTLY'}")
    print(f"{'='*70}\n")



### Test Agent 1 : Require Retrieval

In [19]:
query_agent("What is asynchronous speed?", thread_id="session_1")


👤 User: What is astynchronous speed?

🔍 Agent: [Calling retrieval tool...]
🤖 Agent: Asynchronous speed refers to the operational speed of an induction motor that is lower than the synchronous speed. In electrical machines, synchronous speed (Ns) is the speed at which the magnetic field rotates, determined by the frequency of the electrical supply and the number of poles in the motor. The speed of the rotor in an induction motor is always less than this synchronous speed, which creates a relative motion between the rotating magnetic field and the rotor, allowing for torque generation.

The difference between the synchronous speed and the rotor speed is referred to as "slip." Slip is essential for the operation of induction motors, as it allows the rotor to induce a current and produce torque. Mathematically, slip (s) can be expressed as:

\[ s = \frac{N_s - N_r}{N_s} \]

where \(N_s\) is the synchronous speed and \(N_r\) is the rotor speed. 

In summary, asynchronous speed is the speed

### Test Agent 2 : Require Retrieval

In [22]:
query_agent("What happens if the rotor reaches the speed of the stator, prove it.", thread_id="session_1")


👤 User: What happens if the rotor reaches the speed of the stator, prove it.

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: If the rotor of an induction motor were to reach the synchronous speed of the stator field, several critical issues would arise:

1. **Loss of Induction**: The operation of an induction motor relies on the relative motion between the rotating magnetic field (produced by the stator) and the rotor. This relative motion induces currents in the rotor, which in turn produce torque. If the rotor reaches synchronous speed, there would be no relative motion, resulting in no induced rotor currents. Consequently, the torque would drop to zero.

2. **No Torque Generation**: Since torque is generated due to the interaction of the magnetic field from the stator and the induced current in the rotor, achieving synchronous speed would mean that the rotor cannot generate torque to overcome any load. The mot

### Test Agent 3 : Require Retrieval

In [21]:
query_agent("What is capacitive coupling?", thread_id="session_1")


👤 User: What is capacitive coupling?

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: Capacitive coupling is a phenomenon in electronics where an electric field between two conductive elements, separated by an insulating material (dielectric), allows for the transfer of energy from one circuit to another without a direct electrical connection. This occurs typically through capacitors, which store and release electrical energy.

In practical applications, capacitive coupling is often used in various electronic circuits for signal transmission, filtering, and coupling stages in amplifiers. It enables the passage of AC signals while blocking DC components, making it useful in applications such as audio equipment and RF circuits.

Capacitive coupling can introduce certain effects, such as phase shifts and frequency response alterations, which can be critical in the design and functioning of electronic systems. It is important to consider these effects to

### Test Agent 4: Require Retrieval

In [23]:
query_agent("Summarixe the birth and growth of the IC op.Amps", thread_id="session_1")


👤 User: Summarixe the birth and growth of the IC op.Amps

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: The birth and growth of integrated circuit (IC) operational amplifiers (op-amps) marked a significant advancement in electronics, enabling more efficient and compact circuit designs.

1. **Early Development**: The operational amplifier concept originated in the 1960s, with Fairchild Semiconductor introducing the first IC op-amps, including models 702, 709, and 741, between 1964 and 1968. These devices were revolutionary as they provided a compact and cost-effective solution for performing various analog functions.

2. **Advancements in Technology**: Following the initial introduction, National Semiconductor launched the 101 and 301 op-amps. These ICs transformed the landscape of electronic circuit design by minimizing the size and complexity of circuits. With the developmen

### Test Agent 5: Require Retrieval

In [30]:
query_agent("Tell me about agentic", thread_id="session_2")


👤 User: Tell me about agentic

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: The term "agentic" was not specifically defined or discussed in the retrieved documents. The documents primarily focus on electrical machines and related concepts, such as magnetic flux and the behavior of coils in AC circuits.

If you're looking for a specific definition or context for "agentic," it typically refers to the capacity of individuals to act independently and make their own choices. In psychological and sociological contexts, it relates to the ability to exercise control over one's actions and decisions.

If you need more detailed information or a different context, please let me know!

📊 Decision: USED RETRIEVAL



This is to test and show that the agent will not answer for questions not in the knowledge base and answered based on its on discretion rather than retrieving nonsense from the documents

### Test Agent 1 : NO retrieval needed

In [25]:
query_agent("Hello, how are you doing?", thread_id="session_2")


👤 User: Hello, how are you doing?

🤖 Agent: I'm here to help you with any questions or information you need! What can I assist you with today?

📊 Decision: ANSWERED DIRECTLY



### Test Agent 2 : NO retrieval needed

In [26]:
query_agent("What is your specialization?", thread_id="session_2")


👤 User: What is your specialization?

🤖 Agent: I'm equipped to assist with information related to Electrical machines and Electronics. If you have any specific questions in those areas, feel free to ask!

📊 Decision: ANSWERED DIRECTLY



### Test Agent 3 : NO retrieval needed

In [28]:
query_agent("Thats nice, what are the technologies used to build you?", thread_id="session_2")


👤 User: Thats nice, what are the technologies used to build you?

🤖 Agent: I'm designed to assist with questions and provide information, particularly in specialized areas like Electrical machines and Electronics. If you have specific questions related to those topics, feel free to ask!

📊 Decision: ANSWERED DIRECTLY



### Test Agent 4 : NO retrieval needed

In [32]:
query_agent("My favorite color is yellow", thread_id="session_2")


👤 User: My favorite color is yellow

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: That's great to hear! If you have any questions or need information on a specific topic, especially related to Electrical machines or Electronics, feel free to ask!

📊 Decision: USED RETRIEVAL



### Test Agent 5: NO retrieval needed

In [ ]:
query_agent("Tell me about agentic", thread_id="session_2")